In [15]:
from typing import List, Tuple, Union

from pyshacl import validate
from rdflib import *
from rdflib.collection import Collection

In [16]:
dmop = Namespace('http://www.e-lico.eu/ontologies/dmo/DMOP/DMOP.owl#')
do = Namespace('https://diviloper.dev/ontology#')
di = Namespace('https://diviloper.dev/ontology/implementations#')
ds = Namespace('https://diviloper.dev/ontology/shapes#')
dr = Namespace('https://diviloper.dev/ontology/rules#')
da = Namespace('https://diviloper.dev/ABox#')


def get_graph():
    g = Graph()
    g.bind('dmop', dmop)
    g.bind('do', do)
    g.bind('di', di)
    g.bind('ds', ds)
    g.bind('dr', dr)
    g.bind('dd', da)
    return g

In [17]:
def add_class(graph, nodes):
    l = nodes if isinstance(nodes, list) else [nodes]
    for node in l:
        graph.add((node, RDF.type, OWL.Class))

In [18]:
def add_union(graph, nodes):
    sequence = Collection(graph, BNode(), nodes)
    union = BNode()
    graph.add((union, OWL.unionOf, sequence.uri))
    return union

In [19]:
def add_property(graph, property, domain, range):
    graph.add((property, RDF.type, RDF.Property))
    graph.add((property, RDFS.domain, add_union(graph, domain) if isinstance(domain, list) else domain))
    graph.add((property, RDFS.range, add_union(graph, range) if isinstance(range, list) else range))

In [20]:
ontology = get_graph()

## Classes

In [21]:
classes = [
    do.Data,
    do.Intent,
    do.Problem,
    do.Algorithm,
    do.Workflow,
    do.DataTag,
    do.Step,
    do.Implementation,
    do.Parameter,
    do.ParameterValue,
    do.Transformation
]
for c in classes:
    add_class(ontology, c)

## Object Properties

In [22]:
properties = [
    (do.overData, do.Intent, do.Data),
    (do.tackles, do.Intent, [do.Problem, do.Algorithm]),
    (do.usingParameter, do.Intent, [do.Parameter, do.ParameterValue]),
    (do.subProblemOf, do.Problem, do.Problem),
    (do.solves, [do.Algorithm, do.Workflow], do.Problem),
    (do.createdFor, do.Workflow, do.Intent),
    (do.applies, do.Workflow, do.Algorithm),
    (do.hasStep, do.Workflow, do.Step),
    (do.hasParameter, [do.Workflow, do.Implementation], do.Parameter),
    (do.specifiesInput, [do.Workflow, do.Implementation], do.DataTag),
    (do.specifiesOutput, [do.Workflow, do.Implementation], do.DataTag),
    (do.followedBy, do.Step, do.Step),
    (do.runs, do.Step, [do.Workflow, do.Implementation]),
    (do.hasParameterValue, do.Step, do.ParameterValue),
    (do.forParameter, do.ParameterValue, do.Parameter),
    (do.hasInput, do.Step, do.Data),
    (do.hasOutput, do.Step, do.Data),
    (do.conformsTo, do.Data, do.DataTag),
    (do.implements, do.Implementation, do.Algorithm),
    (do.hasTransformation, do.Implementation, RDF.List),
]
for s, p, o in properties:
    add_property(ontology, s, p, o)

In [23]:
ontology.serialize('ontology.ttl', format='turtle')

<Graph identifier=Ne1782876e291494193ec44ba15e5e061 (<class 'rdflib.graph.Graph'>)>